In [1]:
import pandas as pd
import sqlite3

In [21]:
import sqlite3

# 创建一个名为 example.db 的SQLite数据库
conn = sqlite3.connect('example.db')

# 创建游标对象，用于执行SQL语句
cursor = conn.cursor()

# 创建一个示例表格
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER NOT NULL
    )
''')

# 插入一些数据
cursor.execute("INSERT INTO users (name, age) VALUES ('Alice', 25)")
cursor.execute("INSERT INTO users (name, age) VALUES ('Bob', 30)")

# 提交事务保存数据
conn.commit()

# 关闭连接
conn.close()


In [22]:
# 重新连接到数据库
conn = sqlite3.connect('example.db')

# 使用 pandas 方便读取 SQL 查询结果
import pandas as pd

# 查询数据
query = "SELECT * FROM users"
df = pd.read_sql_query(query, conn)

# 展示数据
print(df)

# 关闭连接
conn.close()


   id   name  age
0   1  Alice   25
1   2    Bob   30
2   3  Alice   25
3   4    Bob   30


为了模拟涵盖SQL所有重要考点的实验，我们可以设计五个实验场景，并使用多个不同的数据集。这些场景可以包含基本操作（如创建表、插入数据）、中高级操作（如联结查询、分组聚合、窗口函数、子查询、排序等）以及复杂操作（如计算各种比率、复杂查询等）。每个实验使用不同的业务场景并针对特定SQL操作进行练习。

数据集设计
我们将创建以下三张表：

users 表：包含用户的基本信息。
products 表：包含产品信息。
transactions 表：包含用户购买产品的交易记录。

--------------------------------------------
users 表
id	name	age	email
1	Alice	25	alice@example.com
2	Bob	30	bob@example.com
3	Charlie	35	charlie@example.com
4	David	40	david@example.com
5	Eve	29	eve@example.com
products 表
id	product_name	price
1	Laptop	1000
2	Smartphone	500
3	Tablet	300
4	Headphones	100
5	Monitor	200
transactions 表
id	user_id	product_id	quantity	transaction_date
1	1	1	1	2024-01-15
2	2	2	2	2024-02-20
3	3	3	1	2024-03-05
4	1	4	4	2024-04-12
5	4	5	1	2024-05-25

实验 1：基本操作 - 创建表、插入数据和简单查询
步骤：
创建 users, products, 和 transactions 表。
插入表格数据。
简单查询：查询所有用户的姓名、年龄和邮箱。



In [4]:
import sqlite3

# 创建并连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 删除已存在的 users 表
cursor.execute('DROP TABLE IF EXISTS users')

# 创建 users 表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER NOT NULL,
        email TEXT
    )
''')

# 创建 products 表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_name TEXT NOT NULL,
        price REAL NOT NULL
    )
''')

# 创建 transactions 表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        transaction_date TEXT NOT NULL,
        FOREIGN KEY(user_id) REFERENCES users(id),
        FOREIGN KEY(product_id) REFERENCES products(id)
    )
''')

# 插入数据到 users 表
users_data = [
    ('Alice', 25, 'alice@example.com'),
    ('Bob', 30, 'bob@example.com'),
    ('Charlie', 35, 'charlie@example.com'),
    ('David', 40, 'david@example.com'),
    ('Eve', 29, 'eve@example.com')
]
cursor.executemany("INSERT INTO users (name, age, email) VALUES (?, ?, ?)", users_data)

# 插入数据到 products 表
products_data = [
    ('Laptop', 1000),
    ('Smartphone', 500),
    ('Tablet', 300),
    ('Headphones', 100),
    ('Monitor', 200)
]
cursor.executemany("INSERT INTO products (product_name, price) VALUES (?, ?)", products_data)

# 插入数据到 transactions 表
transactions_data = [
    (1, 1, 1, '2024-01-15'),
    (2, 2, 2, '2024-02-20'),
    (3, 3, 1, '2024-03-05'),
    (1, 4, 4, '2024-04-12'),
    (4, 5, 1, '2024-05-25')
]
cursor.executemany("INSERT INTO transactions (user_id, product_id, quantity, transaction_date) VALUES (?, ?, ?, ?)", transactions_data)

conn.commit()

# 查询所有用户的信息
cursor.execute("SELECT name, age, email FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()


('Alice', 25, 'alice@example.com')
('Bob', 30, 'bob@example.com')
('Charlie', 35, 'charlie@example.com')
('David', 40, 'david@example.com')
('Eve', 29, 'eve@example.com')


In [7]:
import sqlite3

# 创建并连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 删除已存在的表
cursor.execute('DROP TABLE IF EXISTS transactions')
cursor.execute('DROP TABLE IF EXISTS products')
cursor.execute('DROP TABLE IF EXISTS users')

# 创建 users 表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        age INTEGER NOT NULL,
        email TEXT
    )
''')

# 创建 products 表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        product_name TEXT NOT NULL,
        price REAL NOT NULL
    )
''')

# 创建 transactions 表
cursor.execute('''
    CREATE TABLE IF NOT EXISTS transactions (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        user_id INTEGER NOT NULL,
        product_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        transaction_date TEXT NOT NULL,
        FOREIGN KEY(user_id) REFERENCES users(id),
        FOREIGN KEY(product_id) REFERENCES products(id)
    )
''')

# 插入数据到 users 表
users_data = [
    ('Alice', 25, 'alice@example.com'),
    ('Bob', 30, 'bob@example.com'),
    ('Charlie', 35, 'charlie@example.com'),
    ('David', 40, 'david@example.com'),
    ('Eve', 29, 'eve@example.com'),
    ('Frank', 22, 'frank@example.com'),
    ('Grace', 31, 'grace@example.com'),
    ('Hannah', 28, 'hannah@example.com'),
    ('Ivy', 26, 'ivy@example.com'),
    ('Jack', 45, 'jack@example.com')
]
cursor.executemany("INSERT INTO users (name, age, email) VALUES (?, ?, ?)", users_data)

# 插入数据到 products 表
products_data = [
    ('Laptop', 1000),
    ('Smartphone', 500),
    ('Tablet', 300),
    ('Headphones', 100),
    ('Monitor', 200),
    ('Keyboard', 50),
    ('Mouse', 25),
    ('Printer', 150),
    ('Camera', 800),
    ('Smartwatch', 200)
]
cursor.executemany("INSERT INTO products (product_name, price) VALUES (?, ?)", products_data)

# 插入数据到 transactions 表
transactions_data = [
    (1, 1, 2, '2024-01-15'),  # Alice 购买了 2 个 Laptop
    (2, 2, 1, '2024-02-20'),  # Bob 购买了 1 个 Smartphone
    (3, 3, 1, '2024-03-05'),  # Charlie 购买了 1 个 Tablet
    (4, 4, 4, '2024-04-12'),  # David 购买了 4 个 Headphones
    (5, 5, 1, '2024-05-25'),  # Eve 购买了 1 个 Monitor
    (6, 6, 3, '2024-06-15'),  # Frank 购买了 3 个 Keyboard
    (7, 7, 2, '2024-07-10'),  # Grace 购买了 2 个 Mouse
    (8, 8, 1, '2024-08-20'),  # Hannah 购买了 1 个 Printer
    (9, 9, 1, '2024-09-05'),  # Ivy 购买了 1 个 Camera
    (10, 10, 2, '2024-10-12'), # Jack 购买了 2 个 Smartwatch
    (1, 2, 1, '2024-10-15'),  # Alice 再次购买了 1 个 Smartphone
    (3, 1, 1, '2024-11-20')   # Charlie 购买了 1 个 Laptop
]
cursor.executemany("INSERT INTO transactions (user_id, product_id, quantity, transaction_date) VALUES (?, ?, ?, ?)", transactions_data)

conn.commit()

# 查询所有用户的信息
cursor.execute("SELECT name, age, email FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

conn.close()


('Alice', 25, 'alice@example.com')
('Bob', 30, 'bob@example.com')
('Charlie', 35, 'charlie@example.com')
('David', 40, 'david@example.com')
('Eve', 29, 'eve@example.com')
('Frank', 22, 'frank@example.com')
('Grace', 31, 'grace@example.com')
('Hannah', 28, 'hannah@example.com')
('Ivy', 26, 'ivy@example.com')
('Jack', 45, 'jack@example.com')


In [9]:
import sqlite3

# 创建并连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查询并打印 users 表的数据
print("users")
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

# 查询并打印 transactions 表的数据
print("transactions")
cursor.execute("SELECT * FROM transactions")
rows = cursor.fetchall()
for row in rows:
    print(row)

# 查询并打印 products 表的数据
print("products")
cursor.execute("SELECT * FROM products")  # 查询 products 表
rows = cursor.fetchall()  # 获取查询结果
for row in rows:  # 打印每一行的结果
    print(row)

conn.close()


users
(1, 'Alice', 25, 'alice@example.com')
(2, 'Bob', 30, 'bob@example.com')
(3, 'Charlie', 35, 'charlie@example.com')
(4, 'David', 40, 'david@example.com')
(5, 'Eve', 29, 'eve@example.com')
(6, 'Frank', 22, 'frank@example.com')
(7, 'Grace', 31, 'grace@example.com')
(8, 'Hannah', 28, 'hannah@example.com')
(9, 'Ivy', 26, 'ivy@example.com')
(10, 'Jack', 45, 'jack@example.com')
transactions
(1, 1, 1, 2, '2024-01-15')
(2, 2, 2, 1, '2024-02-20')
(3, 3, 3, 1, '2024-03-05')
(4, 4, 4, 4, '2024-04-12')
(5, 5, 5, 1, '2024-05-25')
(6, 6, 6, 3, '2024-06-15')
(7, 7, 7, 2, '2024-07-10')
(8, 8, 8, 1, '2024-08-20')
(9, 9, 9, 1, '2024-09-05')
(10, 10, 10, 2, '2024-10-12')
(11, 1, 2, 1, '2024-10-15')
(12, 3, 1, 1, '2024-11-20')
products
(1, 'Laptop', 1000.0)
(2, 'Smartphone', 500.0)
(3, 'Tablet', 300.0)
(4, 'Headphones', 100.0)
(5, 'Monitor', 200.0)
(6, 'Keyboard', 50.0)
(7, 'Mouse', 25.0)
(8, 'Printer', 150.0)
(9, 'Camera', 800.0)
(10, 'Smartwatch', 200.0)


实验 2：进阶操作 - 联结查询、分组与排序
步骤：
联结 users 和 transactions 表，查询每个用户的购买记录。
联结 transactions 和 products 表，计算每个用户购买的产品总金额。
对用户按总金额进行降序排序。


In [3]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 1. 查询每个用户的购买记录
cursor.execute('''
    SELECT users.name, products.product_name, transactions.quantity, transactions.transaction_date 
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
''')
rows = cursor.fetchall()
print("用户购买记录：")
for row in rows:
    print(row)

# 2. 计算每个用户的总购买金额
cursor.execute('''
    SELECT users.name, SUM(products.price * transactions.quantity) as total_spent
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    GROUP BY users.name
''')
rows = cursor.fetchall()
print("\n每个用户的总购买金额：")
for row in rows:
    print(row)

# 3. 对用户按总购买金额进行降序排序
cursor.execute('''
    SELECT users.name, SUM(products.price * transactions.quantity) as total_spent
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    GROUP BY users.name
    ORDER BY total_spent DESC
''')
rows = cursor.fetchall()
print("\n按总金额降序排序的用户：")
for row in rows:
    print(row)

conn.close()


用户购买记录：
('Alice', 'Laptop', 1, '2024-01-15')
('Bob', 'Smartphone', 2, '2024-02-20')
('Charlie', 'Tablet', 1, '2024-03-05')
('Alice', 'Headphones', 4, '2024-04-12')
('David', 'Monitor', 1, '2024-05-25')

每个用户的总购买金额：
('Alice', 1400.0)
('Bob', 1000.0)
('Charlie', 300.0)
('David', 200.0)

按总金额降序排序的用户：
('Alice', 1400.0)
('Bob', 1000.0)
('Charlie', 300.0)
('David', 200.0)


实验 3：窗口函数 - 计算累计总金额、排名等
步骤：
使用窗口函数，按用户计算每次交易的累计总金额。
计算每个用户的购买排名。

In [10]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 1. 计算累计总金额
cursor.execute('''
    SELECT users.name, products.product_name, transactions.quantity, 
           SUM(products.price * transactions.quantity) OVER (PARTITION BY users.id ORDER BY transactions.transaction_date) AS running_total
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    ORDER BY users.name, transactions.transaction_date
''')
rows = cursor.fetchall()
print("每个用户的累计总金额：")
for row in rows:
    print(row)

# 2. 计算购买排名
cursor.execute('''
    SELECT users.name, 
           SUM(products.price * transactions.quantity) as total_spent,
           RANK() OVER (ORDER BY SUM(products.price * transactions.quantity) DESC) as spending_rank
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    GROUP BY users.name
''')
rows = cursor.fetchall()
print("\n每个用户的消费排名：")
for row in rows:
    print(row)

conn.close()


每个用户的累计总金额：
('Alice', 'Laptop', 2, 2000.0)
('Alice', 'Smartphone', 1, 2500.0)
('Bob', 'Smartphone', 1, 500.0)
('Charlie', 'Tablet', 1, 300.0)
('Charlie', 'Laptop', 1, 1300.0)
('David', 'Headphones', 4, 400.0)
('Eve', 'Monitor', 1, 200.0)
('Frank', 'Keyboard', 3, 150.0)
('Grace', 'Mouse', 2, 50.0)
('Hannah', 'Printer', 1, 150.0)
('Ivy', 'Camera', 1, 800.0)
('Jack', 'Smartwatch', 2, 400.0)

每个用户的消费排名：
('Alice', 2500.0, 1)
('Charlie', 1300.0, 2)
('Ivy', 800.0, 3)
('Bob', 500.0, 4)
('Jack', 400.0, 5)
('David', 400.0, 5)
('Eve', 200.0, 7)
('Hannah', 150.0, 8)
('Frank', 150.0, 8)
('Grace', 50.0, 10)


实验 4：高级操作 - 复杂子查询、比率计算
步骤：
使用子查询计算每个用户的平均购买金额。
计算每个用户的购买占比（每个用户的总金额与所有用户总金额的比率）。

In [11]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 1. 每个用户的平均购买金额
cursor.execute('''
    SELECT name, AVG(total_spent) as avg_spent
    FROM (
        SELECT users.name, SUM(products.price * transactions.quantity) as total_spent
        FROM users
        JOIN transactions ON users.id = transactions.user_id
        JOIN products ON transactions.product_id = products.id
        GROUP BY users.name
    )
''')
rows = cursor.fetchall()
print("每个用户的平均购买金额：")
for row in rows:
    print(row)

# 2. 计算每个用户的购买占比
cursor.execute('''
    SELECT name, total_spent, 
           ROUND(total_spent * 100.0 / (SELECT SUM(total_spent) FROM (
               SELECT users.name, SUM(products.price * transactions.quantity) as total_spent
               FROM users
               JOIN transactions ON users.id = transactions.user_id
               JOIN products ON transactions.product_id = products.id
               GROUP BY users.name
           )), 2) as percentage
    FROM (
        SELECT users.name, SUM(products.price * transactions.quantity) as total_spent
        FROM users
        JOIN transactions ON users.id = transactions.user_id
        JOIN products ON transactions.product_id = products.id
        GROUP BY users.name
    )
''')
rows = cursor.fetchall()
print("\n每个用户的购买占比：")
for row in rows:
    print(row)

conn.close()


每个用户的平均购买金额：
('Alice', 645.0)

每个用户的购买占比：
('Alice', 2500.0, 38.76)
('Bob', 500.0, 7.75)
('Charlie', 1300.0, 20.16)
('David', 400.0, 6.2)
('Eve', 200.0, 3.1)
('Frank', 150.0, 2.33)
('Grace', 50.0, 0.78)
('Hannah', 150.0, 2.33)
('Ivy', 800.0, 12.4)
('Jack', 400.0, 6.2)


实验 5：综合练习 - 多表联结、复杂排序、分组与子查询
步骤：
查找在2024年消费最多的用户。
按每个用户每月的购买金额进行分组统计。

In [6]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 1. 查找在2024年消费最多的用户
cursor.execute('''
    SELECT users.name, SUM(products.price * transactions.quantity) as total_spent
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    WHERE transactions.transaction_date BETWEEN '2024-01-01' AND '2024-12-31'
    GROUP BY users.name
    ORDER BY total_spent DESC
    LIMIT 1
''')
row = cursor.fetchone()
print("2024年消费最多的用户：", row)

# 2. 按每个用户每月的购买金额进行分组统计
cursor.execute('''
    SELECT users.name, strftime('%Y-%m', transactions.transaction_date) as month, 
           SUM(products.price * transactions.quantity) as monthly_spent
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    GROUP BY users.name, month
    ORDER BY users.name, month
''')
rows = cursor.fetchall()
print("\n每个用户每月的购买金额：")
for row in rows:
    print(row)

conn.close()


2024年消费最多的用户： ('Alice', 1400.0)

每个用户每月的购买金额：
('Alice', '2024-01', 1000.0)
('Alice', '2024-04', 400.0)
('Bob', '2024-02', 1000.0)
('Charlie', '2024-03', 300.0)
('David', '2024-05', 200.0)


In [12]:
import sqlite3

# 连接到数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 1. 获取当前日期和时间
cursor.execute("SELECT DATETIME('now');")
row = cursor.fetchone()
print("当前日期和时间：", row)

# 2. 获取当前日期
cursor.execute("SELECT DATE('now');")
row = cursor.fetchone()
print("当前日期：", row)

# 3. 获取当前时间
cursor.execute("SELECT TIME('now');")
row = cursor.fetchone()
print("当前时间：", row)

# 4. 日期加法（增加 10 天）
cursor.execute("SELECT DATE('2024-01-01', '+10 days');")
row = cursor.fetchone()
print("2024-01-01 增加 10 天后的日期：", row)

# 5. 日期减法（减少 10 天）
cursor.execute("SELECT DATE('2024-01-01', '-10 days');")
row = cursor.fetchone()
print("2024-01-01 减少 10 天后的日期：", row)

# 6. 提取年份
cursor.execute("SELECT STRFTIME('%Y', '2024-10-21');")
row = cursor.fetchone()
print("2024-10-21 的年份：", row)

# 7. 提取月份
cursor.execute("SELECT STRFTIME('%m', '2024-10-21');")
row = cursor.fetchone()
print("2024-10-21 的月份：", row)

# 8. 提取日
cursor.execute("SELECT STRFTIME('%d', '2024-10-21');")
row = cursor.fetchone()
print("2024-10-21 的日：", row)

# 9. 获取星期几（0 = 星期一, 6 = 星期日）
cursor.execute("SELECT STRFTIME('%w', '2024-10-21');")
row = cursor.fetchone()
print("2024-10-21 是星期几（0=星期日, 6=星期六）：", row)

# 10. 日期格式化
cursor.execute("SELECT STRFTIME('%Y-%m-%d', '2024-10-21');")
row = cursor.fetchone()
print("格式化后的日期：", row)

# 11. 获取 Unix 时间戳
cursor.execute("SELECT STRFTIME('%s', '2024-10-21');")
row = cursor.fetchone()
print("2024-10-21 的 Unix 时间戳：", row)

# 12. 当前 Unix 时间戳
cursor.execute("SELECT STRFTIME('%s', 'now');")
row = cursor.fetchone()
print("当前 Unix 时间戳：", row)

# 关闭数据库连接
conn.close()


当前日期和时间： ('2024-10-21 08:40:10',)
当前日期： ('2024-10-21',)
当前时间： ('08:40:10',)
2024-01-01 增加 10 天后的日期： ('2024-01-11',)
2024-01-01 减少 10 天后的日期： ('2023-12-22',)
2024-10-21 的年份： ('2024',)
2024-10-21 的月份： ('10',)
2024-10-21 的日： ('21',)
2024-10-21 是星期几（0=星期日, 6=星期六）： ('1',)
格式化后的日期： ('2024-10-21',)
2024-10-21 的 Unix 时间戳： ('1729468800',)
当前 Unix 时间戳： ('1729500010',)


实验 6：计算每个用户的最后一次购买日期与购买产品
目标：
使用窗口函数 ROW_NUMBER() 找到每个用户的最后一次购买的产品名称和购买日期。

In [17]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 计算每个用户最后一次购买的产品和购买日期
cursor.execute('''
    SELECT name, product_name, transaction_date
    FROM (
        SELECT users.name, products.product_name, transactions.transaction_date,
               ROW_NUMBER() OVER (PARTITION BY users.id ORDER BY transactions.transaction_date DESC) as rn
        FROM users
        JOIN transactions ON users.id = transactions.user_id
        JOIN products ON transactions.product_id = products.id
    ) as t
    WHERE t.rn = 1
''')
rows = cursor.fetchall()
print("每个用户最后一次购买的产品和购买日期：")
for row in rows:
    print(row)

conn.close()


每个用户最后一次购买的产品和购买日期：
('Alice', 'Smartphone', '2024-10-15')
('Bob', 'Smartphone', '2024-02-20')
('Charlie', 'Laptop', '2024-11-20')
('David', 'Headphones', '2024-04-12')
('Eve', 'Monitor', '2024-05-25')
('Frank', 'Keyboard', '2024-06-15')
('Grace', 'Mouse', '2024-07-10')
('Hannah', 'Printer', '2024-08-20')
('Ivy', 'Camera', '2024-09-05')
('Jack', 'Smartwatch', '2024-10-12')


实验 7：查找购买次数超过2次的用户及其购买次数
目标：
使用 HAVING 和 COUNT() 聚合函数查找购买次数超过2次的用户，并显示他们的购买次数。

In [19]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查找购买次数超过2次的用户及其购买次数
cursor.execute('''
    SELECT users.name, COUNT(transactions.id) as purchase_count
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    GROUP BY users.name
    HAVING COUNT(transactions.id) > 1
''')
rows = cursor.fetchall()
print("购买次数超过2次的用户及其购买次数：")
for row in rows:
    print(row)

conn.close()


购买次数超过2次的用户及其购买次数：
('Alice', 2)
('Charlie', 2)


实验 8：查找用户的平均购买金额与总购买金额
目标：
计算每个用户的平均购买金额与总购买金额，并按总金额排序。

In [9]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 计算每个用户的平均购买金额和总购买金额
cursor.execute('''
    SELECT users.name, 
           AVG(products.price * transactions.quantity) as avg_spent, 
           SUM(products.price * transactions.quantity) as total_spent
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    GROUP BY users.name
    ORDER BY total_spent DESC
''')
rows = cursor.fetchall()
print("每个用户的平均购买金额和总购买金额：")
for row in rows:
    print(row)

conn.close()


每个用户的平均购买金额和总购买金额：
('Alice', 700.0, 1400.0)
('Bob', 1000.0, 1000.0)
('Charlie', 300.0, 300.0)
('David', 200.0, 200.0)


实验 9：查找每个用户购买金额占总购买金额的百分比
目标：
计算每个用户的购买金额占所有用户总购买金额的百分比，并按百分比降序排列。

In [10]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 计算每个用户购买金额占总金额的百分比
cursor.execute('''
    SELECT users.name, 
           SUM(products.price * transactions.quantity) as total_spent, 
           ROUND(SUM(products.price * transactions.quantity) * 100.0 / 
                 (SELECT SUM(products.price * transactions.quantity) FROM transactions JOIN products ON transactions.product_id = products.id), 2) as percentage
    FROM users
    JOIN transactions ON users.id = transactions.user_id
    JOIN products ON transactions.product_id = products.id
    GROUP BY users.name
    ORDER BY percentage DESC
''')
rows = cursor.fetchall()
print("每个用户的购买金额占总金额的百分比：")
for row in rows:
    print(row)

conn.close()


每个用户的购买金额占总金额的百分比：
('Alice', 1400.0, 48.28)
('Bob', 1000.0, 34.48)
('Charlie', 300.0, 10.34)
('David', 200.0, 6.9)


实验 10：查找最畅销的产品
目标：
查找购买数量最多的产品，并显示产品名称及其总购买数量。

In [11]:
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查找最畅销的产品及其总购买数量
cursor.execute('''
    SELECT products.product_name, SUM(transactions.quantity) as total_quantity
    FROM products
    JOIN transactions ON products.id = transactions.product_id
    GROUP BY products.product_name
    ORDER BY total_quantity DESC
    LIMIT 1
''')
row = cursor.fetchone()
print("最畅销的产品及其总购买数量：", row)

conn.close()


最畅销的产品及其总购买数量： ('Headphones', 4)


然后是尽量多的函数的运用

In [ ]:
实验 1：查询用户的购买总金额、平均购买单价，并筛选购买总金额大于200的用户
涉及SQL功能：
JOIN：将 users、transactions 和 products 表连接起来。
SUM 和 AVG：聚合函数，分别计算购买总金额和平均单价。
GROUP BY：按用户分组。
HAVING：过滤出购买总金额大于200的用户。


In [13]:
import sqlite3

# 连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查询每个用户的总购买金额和平均购买单价
query = '''
    SELECT 
        u.name, 
        SUM(p.price * t.quantity) AS total_spent, 
        AVG(p.price) AS avg_price
    FROM 
        users u
    JOIN 
        transactions t ON u.id = t.user_id
    JOIN 
        products p ON t.product_id = p.id
    GROUP BY 
        u.name
    HAVING 
        total_spent > 200
'''
cursor.execute(query)
rows = cursor.fetchall()

# 打印结果
for row in rows:
    print(f"User:",row )

conn.close()


User: ('Alice', 1400.0, 550.0)
User: ('Bob', 1000.0, 500.0)
User: ('Charlie', 300.0, 300.0)


实验 2：计算每个产品的销售数量和总收入，并按销售数量排序
涉及SQL功能：
SUM：聚合函数，计算产品销售数量和总收入。
GROUP BY：按产品分组。
ORDER BY：按产品销售数量降序排列。

In [14]:
import sqlite3

# 连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查询每个产品的销售数量和总收入
query = '''
    SELECT 
        p.product_name, 
        SUM(t.quantity) AS total_quantity, 
        SUM(t.quantity * p.price) AS total_revenue
    FROM 
        products p
    JOIN 
        transactions t ON p.id = t.product_id
    GROUP BY 
        p.product_name
    ORDER BY 
        total_quantity DESC
'''
cursor.execute(query)
rows = cursor.fetchall()

# 打印结果
for row in rows:
    print(f"Product: {row[0]}, Total Sold: {row[1]}, Total Revenue: {row[2]}")

conn.close()


Product: Headphones, Total Sold: 4, Total Revenue: 400.0
Product: Smartphone, Total Sold: 2, Total Revenue: 1000.0
Product: Tablet, Total Sold: 1, Total Revenue: 300.0
Product: Monitor, Total Sold: 1, Total Revenue: 200.0
Product: Laptop, Total Sold: 1, Total Revenue: 1000.0


In [ ]:
实验 3：计算用户最近一次交易的日期，并按用户姓名排序
涉及SQL功能：
MAX：聚合函数，找到每个用户的最近交易日期。
GROUP BY：按用户分组。
ORDER BY：按用户姓名升序排列。

In [15]:
import sqlite3

# 连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查询每个用户最近一次交易的日期
query = '''
    SELECT 
        u.name, 
        MAX(t.transaction_date) AS last_transaction_date
    FROM 
        users u
    JOIN 
        transactions t ON u.id = t.user_id
    GROUP BY 
        u.name
    ORDER BY 
        u.name ASC
'''
cursor.execute(query)
rows = cursor.fetchall()

# 打印结果
for row in rows:
    print(f"User: {row[0]}, Last Transaction Date: {row[1]}")

conn.close()


User: Alice, Last Transaction Date: 2024-04-12
User: Bob, Last Transaction Date: 2024-02-20
User: Charlie, Last Transaction Date: 2024-03-05
User: David, Last Transaction Date: 2024-05-25


In [ ]:
实验 4：查询用户总交易额大于特定用户的用户列表（IN 和 EXISTS 子查询、聚合、连接）
涉及SQL功能：
SELECT、WHERE：基本查询。
JOIN：连接查询，连接 users、transactions 和 products。
IN 和 EXISTS 子查询：用于找到特定用户的总交易额。
HAVING：过滤总交易额大于某个值的分组结果。
GROUP BY 和 ORDER BY：按用户分组，并按总交易额排序。

In [16]:
import sqlite3

# 连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查询总交易额大于用户 'Alice' 的其他用户
query = '''
    SELECT u.name, SUM(t.quantity * p.price) AS total_spent
    FROM users u
    JOIN transactions t ON u.id = t.user_id
    JOIN products p ON t.product_id = p.id
    WHERE u.id IN (
        SELECT user_id 
        FROM transactions t2
        JOIN products p2 ON t2.product_id = p2.id
        WHERE t2.user_id = (SELECT id FROM users WHERE name = 'Alice')
    )
    GROUP BY u.name
    HAVING total_spent > (
        SELECT SUM(t.quantity * p.price)
        FROM transactions t
        JOIN products p ON t.product_id = p.id
        WHERE t.user_id = (SELECT id FROM users WHERE name = 'Alice')
    )
    ORDER BY total_spent DESC
'''
cursor.execute(query)
rows = cursor.fetchall()

# 打印结果
for row in rows:
    print(f"User: {row[0]}, Total Spent: {row[1]}")

conn.close()


In [ ]:
实验 5：查询两个用户购买相同商品的商品名称及价格（INTERSECT 集合操作、连接查询）
涉及SQL功能：
INTERSECT：集合操作，找出两个用户购买相同的商品。
SELECT 和 WHERE：基本查询。
子查询：用于过滤两个用户购买的商品。

In [17]:
import sqlite3

# 连接数据库
conn = sqlite3.connect('example.db')
cursor = conn.cursor()

# 查询 'Alice' 和 'Bob' 都购买过的商品名称及价格
query = '''
    SELECT product_name, price 
    FROM products
    WHERE id IN (
        SELECT product_id FROM transactions WHERE user_id = (SELECT id FROM users WHERE name = 'Alice')
    )
    INTERSECT
    SELECT product_name, price 
    FROM products
    WHERE id IN (
        SELECT product_id FROM transactions WHERE user_id = (SELECT id FROM users WHERE name = 'Bob')
    )
'''
cursor.execute(query)
rows = cursor.fetchall()

# 打印结果
for row in rows:
    print(f"Product: {row[0]}, Price: {row[1]}")

conn.close()


In [ ]:
关键功能解释：
基本查询：

SELECT：选择要查询的列。
WHERE：用于过滤条件，如用户名称、价格等。
GROUP BY 和 HAVING：用于对数据分组，并对聚合结果进行过滤。
ORDER BY：对查询结果进行排序。
连接查询：

JOIN：将多个表进行连接，能够从多个数据源中提取关联信息。
嵌套查询：

IN 和 EXISTS：用于嵌套查询，从子查询中提取符合条件的数据。
子查询与运算符：通过子查询结果与主查询进行比较，如上面的 EXCEPT 和 INTERSECT。
集合查询：

UNION、INTERSECT、EXCEPT：这些操作可以将多个查询结果集进行并集、交集和差集操作。